In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from stable_baselines3.common.vec_env import DummyVecEnv
# from sb3_contrib import RecurrentPPO
# from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
import random
# from stable_baselines3.common.vec_env import SubprocVecEnv
# import matplotlib.pyplot as plt
# from typing import Tuple, Optional
import math
import mplfinance as mpf
from readAndSortCsv import read_and_sort_csv
from createSequences import create_sequences
from labelSequences import precompute_label_info, get_labels_from_precomputed
from plotData import plot_input_output_combined, plot_input_output_combined_with_label
from trades import compute_profit, compute_sharpe_ratio
from prepareScaledData import scale_X_0_1, encode_labels, compute_sample_profit, get_profitable_indices, get_flat_indices


In [2]:
MODEL_PATH = 'simple1dcnn_state_dict.pth'
required_columns = ['date', 'open', 'high', 'low', 'close', 'volume']
file_path = r"E:\AICore\Data\Binance_BTCUSDT_2024_minute.csv"
input_window = 600
output_window = 60
np.set_printoptions(formatter={'float_kind': lambda x: f'{x:.2f}'})


# Read data and compute X, y, labels

In [3]:
# # 1) Read & sort in descending order
# df = read_and_sort_csv(file_path, required_columns)
# print(f"Successfully read {len(df)} rows from the CSV file.")
# print("First few rows (descending):")
# print(df.head(3))
# print("Last few rows (descending):")
# print(df.tail(3))

# # 2) Create input/output sequences
# X, y = create_sequences(df, input_window=input_window, output_window=output_window)

In [4]:
# # labels = label_sequences(X, y, close_idx=3, volume_idx=4, alpha=0.7, beta=0.3, threshold=1.0)
# (
#     direction_pct_arr,
#     range_diff_arr,
#     p_val_arr,
#     median_in_arr,
#     median_out_arr
# ) = precompute_label_info(X, y, close_idx=3, high_idx=1, low_idx=2, volume_idx=4)

# labels = get_labels_from_precomputed(
#     direction_pct_arr, range_diff_arr, p_val_arr, median_in_arr, median_out_arr,
#     alpha=1.469647214304428, beta=0.0054738562416353255, gamma=0, threshold=0.1426995297068393
# )
# #  {'alpha': 1.469647214304428, 'beta': 0.0054738562416353255, 'threshold': 0.1426995297068393}

# print(f"\nCreated {X.shape[0]} input-output sequences.")
# print(f"Input shape: {X.shape}")
# print(f"Output shape: {y.shape}")

# # 3) (Optional) Scale sequences ...

# # 4) (Optional) Save sequences ...

# # 5) Plot a sample input-output window on one chart

In [5]:
# np.save('X.npy', X)
# np.save('y.npy', y)
# np.save('labels.npy', labels)

# Load X, y, labels

In [6]:
X = np.load('X.npy')
y = np.load('y.npy')
labels = np.load('labels.npy',allow_pickle = True)

# Tuning

In [ ]:
# import optuna

# # Precompute once
# precomp = precompute_label_info(X, y, close_idx=3, high_idx=1, low_idx=2, volume_idx=4)

# def objective(trial, objective_name="profit"):
#     """
#     objective_name can be "profit" or "sharpe" to switch the metric.
#     """
#     # Example: fix alpha = 0, or let it vary
#     # alpha = 0
#     alpha = trial.suggest_float("alpha", -2.0, 2.0)
    
#     beta = trial.suggest_float("beta", -1.0, 1.0)
#     # gamma = trial.suggest_float("gamma", 0.0, 1.0)
#     gamma = 0
#     threshold = trial.suggest_float("threshold", 0.0, 1.0)

#     # Unpack precomputed
#     direction_pct_arr, range_diff_arr, p_val_arr, median_in_arr, median_out_arr = precomp
    
#     # Get labels quickly
#     labels = get_labels_from_precomputed(
#         direction_pct_arr, range_diff_arr, p_val_arr, median_in_arr, median_out_arr,
#         alpha=alpha, beta=beta, gamma=gamma, threshold=threshold
#     )
    
#     # Evaluate performance
#     if objective_name == "sharpe":
#         score = compute_sharpe_ratio(X, y, labels)
#         if score == 0: score = -10
#     else:
#         score = compute_profit(X, y, labels)
    
#     # We want to maximize the metric => minimize the negative
#     return -score

# # Create and run a study, e.g. for profit
# study = optuna.create_study(direction="minimize")
# study.optimize(lambda t: objective(t, objective_name="profit"), n_trials=500)

# print("==== Profit Objective ====")
# print("Best params:", study.best_params)
# print("Max profit found:", -study.best_value)




In [ ]:
# # If you want a separate run for Sharpe ratio
# study_sharpe = optuna.create_study(direction="minimize")
# study_sharpe.optimize(lambda t: objective(t, objective_name="sharpe"), n_trials=500)

# print("==== Sharpe Objective ====")
# print("Best params:", study_sharpe.best_params)
# print("Max Sharpe found:", -study_sharpe.best_value)

# Plot raw data with lable

In [ ]:
sample_idx = 4
print(labels[sample_idx])
# plot_input_output_combined(
#     df_original=df,
#     start_idx=sample_idx,
#     input_window=input_window,
#     output_window=output_window,
#     title="Sample Input & Output on One Chart (start_idx=0)"
# )
plot_input_output_combined_with_label(
    X[sample_idx],
    y[sample_idx],
    label=labels[sample_idx],
    title="Input+Output, Colored by Label"
)

In [ ]:
labels[0]

In [ ]:
unique, counts = np.unique(labels, return_counts=True)

dict(zip(unique, counts))


In [ ]:
print(np.where(labels == 'long'))
print(np.where(labels == 'short'))
print(np.where(labels == 'flat'))


# Prepare and scale data

In [13]:
oaoao_long = []
oaoao_short = []
idxs = get_profitable_indices(X, y, labels)
profit_lables= labels[idxs]
for i, v in enumerate(idxs):
    if labels[v] == "short":
        oaoao_short.append((v))
    elif labels[v] == "long":
        oaoao_long.append((v))
flat_idxs = get_flat_indices(labels, 80000)
oaoao_flat = random.sample(flat_idxs, 80000)

In [ ]:
print(len(oaoao_long))
print(len(oaoao_short))
print(len(oaoao_flat))

In [15]:
data_idxs = oaoao_long[:80000] + oaoao_short[:80000] + oaoao_flat

In [ ]:
compute_profit(X[data_idxs], y[data_idxs], labels[data_idxs])

In [ ]:
np.random.shuffle(data_idxs)
zipped_data = list(zip(X[data_idxs], labels[data_idxs], y[data_idxs]))

# 3) Slice into train/validate/test
#    (These slice boundaries are your choice)
train_slice = zipped_data[:192000]
val_slice   = zipped_data[192000:-24000]
test_slice  = zipped_data[192000 + 24000:]

# 4) "Unzip" back into separate arrays
X_train, y_train, out_train = zip(*train_slice)
X_val,   y_val, out_val   = zip(*val_slice)
X_test,  y_test, out_test  = zip(*test_slice)

# 5) Convert each to np.array
X_train, y_train, out_train = np.array(X_train), np.array(y_train), np.array(out_train)
X_val,   y_val, out_val    = np.array(X_val),   np.array(y_val), np.array(out_val)
X_test,  y_test, out_test  = np.array(X_test),  np.array(y_test), np.array(out_test)

In [ ]:
# Suppose we have:
# X_train, X_val, X_test as (num_samples, input_window, num_features)
# y_train, y_val, y_test as string arrays of shape (num_samples,)

# 1) Scale X's
X_train_scaled, scaler_train = scale_X_0_1(X_train)
X_val_scaled = scaler_train.transform(X_val.reshape(-1, X_val.shape[-1])) \
                           .reshape(X_val.shape)
X_test_scaled = scaler_train.transform(X_test.reshape(-1, X_test.shape[-1])) \
                            .reshape(X_test.shape)

# 2) Encode labels
y_train_encoded = encode_labels(y_train)
y_val_encoded   = encode_labels(y_val)
y_test_encoded  = encode_labels(y_test)

print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_encoded shape:", y_train_encoded.shape)
print("Sample encoded labels:", np.unique(y_train_encoded))


# Prepare model

In [19]:
import numpy as np

# Suppose X_train_scaled.shape = (19000, 600, 5)
# and y_train_encoded.shape = (19000,)
X_train_transposed = np.transpose(X_train_scaled, (0, 2, 1))  # (19000, 5, 600)

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        # Convert numpy arrays to PyTorch tensors
        self.X = torch.from_numpy(X).float()  # shape: (num_samples, channels, seq_len)
        self.y = torch.from_numpy(y).long()   # shape: (num_samples,)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        # Return (features, label) for sample 'idx'
        return self.X[idx], self.y[idx]

train_dataset = TimeSeriesDataset(X_train_transposed, y_train_encoded)

batch_size = 600
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [20]:

class Simple1DCNN(nn.Module):
    def __init__(self, num_channels=5, seq_len=600, num_classes=3, dropout_p=0.3):
        """
        An improved 1D CNN for time-series classification.

        Args:
            num_channels (int): Number of input channels (features). Defaults to 5 (e.g., open/high/low/close/volume).
            seq_len (int): Number of timesteps in each sample (e.g., 600).
            num_classes (int): Number of output classes (e.g., 3 for 'short','flat','long').
            dropout_p (float): Dropout probability for regularization. Defaults to 0.3.
        """
        super(Simple1DCNN, self).__init__()
        
        # -------------------------
        # Convolution Block 1
        # -------------------------
        self.conv1 = nn.Conv1d(
            in_channels=num_channels, 
            out_channels=32,
            kernel_size=5,         # Larger kernel for broader context
            stride=1, 
            padding=2              # "same" padding
        )
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=2)  # Halves sequence length: 600 -> 300

        # -------------------------
        # Convolution Block 2
        # -------------------------
        self.conv2 = nn.Conv1d(
            in_channels=32, 
            out_channels=64,
            kernel_size=5, 
            stride=1, 
            padding=2
        )
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2)  # 300 -> 150

        # -------------------------
        # Convolution Block 3
        # -------------------------
        self.conv3 = nn.Conv1d(
            in_channels=64, 
            out_channels=128,
            kernel_size=3, 
            stride=1, 
            padding=1
        )
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=2)  # 150 -> 75

        # After 3 poolings, seq_len -> seq_len / 8
        # so final sequence length = 600 / 2 / 2 / 2 = 75
        # out channels = 128
        # flattened size = 128 * 75 = 9600
        self.dropout = nn.Dropout(p=dropout_p)

        self.fc = nn.Linear(128 * (seq_len // 8), num_classes)

        # OPTIONAL: If you want to do global average pooling (instead of flattening),
        # you can comment out the above fc dimension logic and do:
        # self.global_pool = nn.AdaptiveAvgPool1d(1)
        # self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        """
        x shape: (batch_size, channels=num_channels, seq_len)
        """
        # -------------------------
        # Block 1
        # -------------------------
        x = self.conv1(x)    # (batch, 32, seq_len=600)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)    # (batch, 32, 300)

        # -------------------------
        # Block 2
        # -------------------------
        x = self.conv2(x)    # (batch, 64, 300)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)    # (batch, 64, 150)

        # -------------------------
        # Block 3
        # -------------------------
        x = self.conv3(x)    # (batch, 128, 150)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)    # (batch, 128, 75)

        # -------------------------
        # Flatten or Global Pool
        # -------------------------
        # 1) Flatten approach
        x = x.view(x.size(0), -1)  # => (batch, 128 * 75)
        
        # 2) Dropout for regularization
        x = self.dropout(x)

        # 3) Fully connected output
        x = self.fc(x)  # => (batch, num_classes=3)

        # OPTIONAL (Global Pooling) approach:
        # x = self.global_pool(x)  # => (batch, 128, 1)
        # x = x.squeeze(-1)        # => (batch, 128)
        # x = self.dropout(x)
        # x = self.fc(x)

        return x

model = Simple1DCNN(num_channels=5, seq_len=600, num_classes=3)


# Load model

In [ ]:
# Reconstruct the model architecture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Simple1DCNN(num_channels=5, seq_len=600, num_classes=3)

# Load the saved state dictionary
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))

# Move to device
model.to(device)


# Train model

In [22]:
criterion = nn.CrossEntropyLoss()

In [23]:

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# epoch = 0
# while epoch_loss > 0.1:
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0

#     for X_batch, y_batch in train_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
#         optimizer.zero_grad()
        
#         outputs = model(X_batch)
#         loss = criterion(outputs, y_batch)
        
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item() * X_batch.size(0)
#         _, pred = torch.max(outputs, 1)
#         correct += (pred == y_batch).sum().item()
#         total += y_batch.size(0)
    
#     epoch_loss = running_loss / total
#     epoch_acc = correct / total
#     print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")


In [24]:
# torch.save(model.state_dict(), MODEL_PATH)
# print(f"Final model saved to {MODEL_PATH}")

# Validate model

In [25]:
# Suppose X_val_scaled.shape = (val_size, 600, 5)
# Suppose y_val_encoded.shape = (val_size,)

# 1) Transpose
X_val_transposed = np.transpose(X_val_scaled, (0, 2, 1))  # shape: (val_size, 5, 600)

# 2) Wrap in a Dataset
val_dataset = TimeSeriesDataset(X_val_transposed, y_val_encoded)

# 3) Create DataLoader (batch_size can match or differ from train)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

model.eval()
val_loss = 0.0
correct = 0
total = 0

# To collect predictions and true labels for further metrics
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model(X_batch)              # shape: (batch_size, num_classes)
        loss = criterion(outputs, y_batch)

        val_loss += loss.item() * X_batch.size(0)

        # Predictions
        _, predicted = torch.max(outputs, 1)  # shape: (batch_size,)
        
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
        
        # Store predictions & labels for confusion matrix, etc.
        all_preds.append(predicted.cpu().numpy())
        all_labels.append(y_batch.cpu().numpy())

# Convert lists of arrays into a single 1D array
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

val_loss /= total
val_acc = correct / total

print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}")

# --------------------------------------------------------------------------
# Additional Metrics
# --------------------------------------------------------------------------

# 1) Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# 2) Classification Report
#    If you have 3 classes: 0="short", 1="flat", 2="long" (example)
target_names = ["short", "flat", "long"]  # adjust if needed
report = classification_report(all_labels, all_preds, target_names=target_names)
print("Classification Report:")
print(report)


# Test model

In [37]:
# Suppose X_test_scaled.shape = (test_size, 600, 5)
# Suppose y_test_encoded.shape = (test_size,)

# 1) Transpose
X_test_transposed = np.transpose(X_test_scaled, (0, 2, 1))  # shape: (test_size, 5, 600)

# 2) Wrap in a Dataset
test_dataset = TimeSeriesDataset(X_test_transposed, y_test_encoded)

# 3) Create DataLoader (batch_size can match or differ from train)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Let's do inference on a single sample from X_test
X_single = X_test[-24000]  # shape (600,5) as an example
out_single = out_test[-24000]
lable_single = y_test[-24000]
X_single_scaled = scaler_train.transform(X_single)  # shape (600,5)
X_single_scaled = np.expand_dims(X_single_scaled, axis=0)  # => (1,600,5)
X_single_transposed = np.transpose(X_single_scaled, (0, 2, 1))  # => (1,5,600)
X_single_tensor = torch.from_numpy(X_single_transposed).float().to(device)

model.eval()
with torch.no_grad():
    output = model(X_single_tensor)   # => shape (1,3)
    _, predicted = torch.max(output, 1)
    predicted_label = predicted.item()  # 0,1,2

label_map = {0:"short",1:"flat",2:"long"}
print("Single sample predicted label:", label_map[predicted_label])

In [ ]:
# Let's do inference on a single sample from X_test
predicted_labels = []
for X_batch, y_batch  in train_loader:
    model.eval()
    with torch.no_grad():
        output = model(X_batch)   # => shape (1,3)
        _, predicted = torch.max(output, 1)
        predicted_label = predicted.item()  # 0,1,2

    label_map = {0:"short",1:"flat",2:"long"}
    predicted_labels.append(label_map[predicted_label])

In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

model.eval()
val_loss = 0.0
correct = 0
total = 0

# To collect predictions and true labels for further metrics
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model(X_batch)              # shape: (batch_size, num_classes)
        loss = criterion(outputs, y_batch)

        val_loss += loss.item() * X_batch.size(0)

        # Predictions
        _, predicted = torch.max(outputs, 1)  # shape: (batch_size,)
        
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
        
        # Store predictions & labels for confusion matrix, etc.
        all_preds.append(predicted.cpu().numpy())
        all_labels.append(y_batch.cpu().numpy())

# Convert lists of arrays into a single 1D array
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

val_loss /= total
val_acc = correct / total

print(f"Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}")

# --------------------------------------------------------------------------
# Additional Metrics
# --------------------------------------------------------------------------

# 1) Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# 2) Classification Report
#    If you have 3 classes: 0="short", 1="flat", 2="long" (example)
target_names = ["short", "flat", "long"]  # adjust if needed
report = classification_report(all_labels, all_preds, target_names=target_names)
print("Classification Report:")
print(report)


In [ ]:
all_preds

In [ ]:
kek = labels[data_idxs[:192000]] == predicted_labels
unique, counts = np.unique(kek, return_counts=True)

dict(zip(unique, counts))

In [ ]:
compute_profit(X[data_idxs[:192000]], y[data_idxs[:192000]], predicted_labels)
# compute_profit(X, y, predicted_labels)


In [ ]:
compute_profit(X[data_idxs[:192000]], y[data_idxs[:192000]], labels[data_idxs[:192000]])
# compute_profit(X, y, labels)


In [ ]:
np.unique(y_v == labels[data_idxs[-24000:]])

In [ ]:
idx = -23200
sample_idx = data_idxs[idx]
print(labels[sample_idx])
# plot_input_output_combined(
#     df_original=df,
#     start_idx=sample_idx,
#     input_window=input_window,
#     output_window=output_window,
#     title="Sample Input & Output on One Chart (start_idx=0)"
# )
plot_input_output_combined_with_label(
    X[sample_idx],
    y[sample_idx],
    label=predicted_labels[idx],
    title="Input+Output, Colored by Label"
)

In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Define Label Encoding/Decoding
label_encoder = LabelEncoder()
label_encoder.fit(["short", "flat", "long"])  # Fit with your label classes

def encode_labels(y):
    return label_encoder.transform(y)

def decode_labels(encoded_label):
    return label_encoder.inverse_transform([encoded_label])[0]

# Inference Function (as defined earlier)
def predict_single(model, scaler, input_data, device, encode_fn, decode_fn=None):
    model.eval()
    input_scaled = scaler.transform(input_data.reshape(-1, input_data.shape[-1])).reshape(input_data.shape)
    input_transposed = np.transpose(input_scaled, (1, 0))
    input_tensor = torch.from_numpy(input_transposed).float().unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
    prediction_encoded = predicted.item()
    if decode_fn:
        prediction = decode_fn(prediction_encoded)
    else:
        prediction = prediction_encoded
    return prediction

def evaluate_new_samples(model, scaler, new_Xs, new_labels, device, encode_fn, decode_fn=None):
    results = []
    for idx, input_data in enumerate(new_Xs):
        prediction = predict_single(model, scaler, input_data, device, encode_fn, decode_fn)
        if new_labels:
            true_label = new_labels[idx]
            correct = (prediction == true_label) if decode_fn else (prediction == encode_fn([true_label])[0])
            results.append({
                'input_index': idx,
                'prediction': prediction,
                'true_label': true_label,
                'correct': correct
            })
        else:
            results.append({
                'input_index': idx,
                'prediction': prediction
            })
    return results



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load or define your trained model
# For example:
# model = YourModel()
# model.load_state_dict(torch.load('model.pth'))
# model.to(device)

y = np.load('y.npy')
# Example new data (replace with your actual data)
new_Xs = np.load('X.npy')

new_labels = np.load('labels.npy',allow_pickle = True)

# Evaluate
results = evaluate_new_samples(
    model=model,
    scaler=scaler_train,
    new_Xs=new_Xs,
    new_labels=new_labels,
    device=device,
    encode_fn=encode_labels,
    decode_fn=decode_labels
)

# Display Results
for res in results:
    print(f"Sample {res['input_index']}: Prediction = {res['prediction']}, "
            f"True Label = {res['true_label']}, Correct = {res['correct']}")